In [16]:
import pandas as pd
import numpy as np

# Set the random seed for reproducibility
np.random.seed(42)

# List of different sizes for the datasets
sizes = [10000, 100000, 1000000, 10000000]
num_features = 5

# Loop through each size and generate, then save the data
for size in sizes:
    # Generate random data
    data = pd.DataFrame(np.random.randn(size, num_features), columns=[f'feature_{i}' for i in range(num_features)])

    # Save the data to a CSV file
    filename = f'random_data_{size}.csv'
    data.to_csv(filename, index=False)

    print(f'Saved {size} samples to {filename}')

Saved 10000 samples to random_data_10000.csv
Saved 100000 samples to random_data_100000.csv
Saved 1000000 samples to random_data_1000000.csv
Saved 10000000 samples to random_data_10000000.csv


In [50]:
%%writefile kmeans_cpu.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>
#include <string.h>

#define MAX_ITER 10
#define MAX_LINE_LENGTH 1024

// Function to read CSV data
int readCSVData(const char *filename, float **data, int *num_points, int *num_dims) {
    FILE *file = fopen(filename, "r");
    if (!file) {
        perror("Unable to open file");
        return -1;
    }

    char line[MAX_LINE_LENGTH];
    int n = 0, d = 0;

    // Read first line to determine the number of dimensions
    if (fgets(line, MAX_LINE_LENGTH, file)) {
        char *token = strtok(line, ",");
        while (token) {
            d++;
            token = strtok(NULL, ",");
        }
    }

    // Count number of points
    while (fgets(line, MAX_LINE_LENGTH, file)) {
        n++;
    }

    // Allocate memory for data
    *data = (float *)malloc(n * d * sizeof(float));
    if (!*data) {
        perror("Unable to allocate memory");
        fclose(file);
        return -1;
    }

    rewind(file);

    // Read data into the array
    int point = 0;
    while (fgets(line, MAX_LINE_LENGTH, file)) {
        char *token = strtok(line, ",");
        int dim = 0;
        while (token) {
            (*data)[point * d + dim] = atof(token);
            token = strtok(NULL, ",");
            dim++;
        }
        point++;
    }

    fclose(file);

    *num_points = n;
    *num_dims = d;

    return 0;
}

float distance(float *a, float *b, int dims) {
    float dist = 0;
    for (int i = 0; i < dims; ++i) {
        dist += (a[i] - b[i]) * (a[i] - b[i]);
    }
    return sqrt(dist);
}

void kMeansClusterAssignment(float *datapoints, int *clust_assn, float *centroids, int N, int K, int D) {
    for (int p = 0; p < N; ++p) {
        float min_dist = INFINITY;
        int closest_centroid = 0;

        for (int c = 0; c < K; ++c) {
            float dist = distance(&datapoints[p * D], &centroids[c * D], D);
            if (dist < min_dist) {
                min_dist = dist;
                closest_centroid = c;
            }
        }
        clust_assn[p] = closest_centroid;
    }
}

void kMeansCentroidUpdate(float *datapoints, int *clust_assn, float *centroids, int *clust_sizes, int N, int K, int D) {
    // Reset centroids and sizes
    for (int c = 0; c < K; ++c) {
        clust_sizes[c] = 0;
        for (int d = 0; d < D; ++d) {
            centroids[c * D + d] = 0;
        }
    }

    // Sum all points in each cluster
    for (int p = 0; p < N; ++p) {
        int cluster_id = clust_assn[p];
        for (int d = 0; d < D; ++d) {
            centroids[cluster_id * D + d] += datapoints[p * D + d];
        }
        clust_sizes[cluster_id] += 1;
    }

    // Normalize the centroids
    for (int c = 0; c < K; ++c) {
        if (clust_sizes[c] > 0) {
            for (int d = 0; d < D; ++d) {
                centroids[c * D + d] /= clust_sizes[c];
            }
        }
    }
}

int main(int argc, char **argv) {
    if (argc != 3) {
        fprintf(stderr, "Usage: %s <input.csv> <K>\n", argv[0]);
        return -1;
    }

    const char *filename = argv[1];
    int K = atoi(argv[2]);

    float *datapoints;
    int *clust_assn;
    float *centroids;
    int *clust_sizes;
    int num_points = 0;
    int num_dims = 0;

    if (readCSVData(filename, &datapoints, &num_points, &num_dims) != 0) {
        fprintf(stderr, "Error reading data from file\n");
        return -1;
    }

    clust_assn = (int *)malloc(num_points * sizeof(int));
    centroids = (float *)malloc(K * num_dims * sizeof(float));
    clust_sizes = (int *)malloc(K * sizeof(int));

    // Initialize centroids (choose first K points as initial centroids)
    for (int c = 0; c < K; ++c) {
        for (int d = 0; d < num_dims; ++d) {
            centroids[c * num_dims + d] = datapoints[c * num_dims + d];
        }
        clust_sizes[c] = 0;
    }

    clock_t start_total = clock();

    for (int cur_iter = 0; cur_iter < MAX_ITER; ++cur_iter) {
        kMeansClusterAssignment(datapoints, clust_assn, centroids, num_points, K, num_dims);
        kMeansCentroidUpdate(datapoints, clust_assn, centroids, clust_sizes, num_points, K, num_dims);

        printf("Iteration %d centroids:\n", cur_iter + 1);
        for (int i = 0; i < K; ++i) {
            printf("Centroid %d: ", i);
            for (int j = 0; j < num_dims; ++j) {
                printf("%f ", centroids[i * num_dims + j]);
            }
            printf("\n");
        }
    }

    clock_t stop_total = clock();
    double total_time = (double)(stop_total - start_total) / CLOCKS_PER_SEC * 1000;
    printf("Total time for K-means clustering: %f milliseconds\n", total_time);

    free(datapoints);
    free(clust_assn);
    free(centroids);
    free(clust_sizes);

    return 0;
}


Overwriting kmeans_cpu.c


In [52]:

!g++ kmeans_cpu.c -o kmeans_cpu.out
!./kmeans_cpu.out random_data_10000000.csv 10

Iteration 1 centroids:
Centroid 0: -0.238239 0.113548 -0.298271 0.230856 0.073452 
Centroid 1: 1.405332 -0.396175 0.798659 -0.398114 0.010792 
Centroid 2: -0.576948 -0.756396 -0.846146 0.330101 -0.806014 
Centroid 3: -0.383057 -0.463510 0.786328 -0.934986 -0.728412 
Centroid 4: 0.607362 -0.627173 -0.456039 0.384967 0.869846 
Centroid 5: 0.405768 0.991291 0.801037 1.019352 -0.121068 
Centroid 6: 0.751024 0.041786 -0.689473 -1.487218 0.680537 
Centroid 7: 0.038314 2.207464 -0.329835 -0.246732 -0.411485 
Centroid 8: -1.087094 0.960262 0.412575 -0.070785 -0.699579 
Centroid 9: -0.024432 0.625252 0.984856 -0.126743 0.957480 
Iteration 2 centroids:
Centroid 0: -0.408781 0.176395 -0.464092 0.314749 0.210014 
Centroid 1: 1.363312 -0.425848 0.691198 -0.253428 -0.213080 
Centroid 2: -0.472094 -0.784854 -0.831681 0.369529 -0.832221 
Centroid 3: -0.395825 -0.586122 0.743331 -0.867919 -0.659911 
Centroid 4: 0.526683 -0.737962 -0.485225 0.549240 0.911190 
Centroid 5: 0.455174 0.713891 0.676081 1.201